# Trabalho final para a matéria: "Anotação de Textos e Processamento de Linguagem Natural (CIC0087)" 😊

## Reconhecimento de determinadas Entidades Nomeadas providas no corpo de contratos.

Extrato de contrato, extrato de convênio, aditamento de extrato de contrato, aviso de licitação, aviso de suspensão de licitação, aviso de revogação/anulação de licitação. 

## Sim, me dei conta de que o pedido da tarefa originalmente era a classificação de algumas entidades definidas, mas tentei com meus conhecimentos tentar classificar tudo para ver o que aconteceria, como um treinamento para a execução da classificação final.

### 👨‍💻 Aluno: Guilherme Braga (17/0162290)

#### 🐱 [Repositório no Github](https://github.com/gui1080/IA_CIC0087_Reconhecimento_de_Entidades_Nomeadas)

### Pegando os dados originais 🧐

In [528]:
from google.colab import files

# importar o arquivo de disco local, com o nome original "contratos.csv"
uploaded = files.upload()

Saving contratos.csv to contratos (21).csv


### Importando, separando e limpando os dados 🚮


In [529]:
import pandas as pd
import numpy as np
import io
from sklearn.utils import shuffle

# Adicionando em dataframe
raw_df = pd.read_csv(io.BytesIO(uploaded['contratos.csv']))
raw_df = raw_df.replace([np.inf, -np.inf], np.nan)
raw_df = raw_df.sample(frac=1).reset_index(drop=True)
raw_df = raw_df.reset_index()

raw_df

,index,id_dodf,tipo_rel,id_rel,anotador_rel,timestamp_rel,tipo_ent,id_ent,anotador_ent,timestamp_ent,offset,length,texto
0,0,1_31.5.2019,REL_AVISO_LICITACAO,R24,vinicius_borges,2022-04-06T12:59:59Z,sistema_compras,304,vinicius_borges,2022-04-06T12:59:27Z,1324565,21,www.comprasnet.gov.br
1,1,12_22.11.2018,REL_ADITAMENTO_CONTRATO,R15,micael_filipe,2022-04-03T11:46:36Z,EXTRATO_ADITAMENTO_CONTRATUAL,295,micael_filipe,2022-02-22T22:11:01Z,601406,518,EXTRATO DE ADITIVO\n3 Termo Aditivo ao Contrat...
2,2,7_2.8.2019,REL_EXTRATO_CONTRATO,R2,alexandre_santana,2022-03-08T11:36:58Z,data_assinatura_contrato,83,alexandre_santana,2022-02-22T02:19:49Z,386367,10,19/07/2019
3,3,5_16.7.2019,REL_EXTRATO_CONTRATO,R4,alexandre_santana,2022-03-08T02:26:00Z,processo_gdf,61,alexandre_santana,2022-02-20T20:54:41Z,622794,22,00196-00000597/2019-70
4,4,24_29.1.2019,REL_ADITAMENTO_CONTRATO,R3,matheus_santos,2022-04-02T23:03:19Z,numero_termo_aditivo,17,matheus_santos,2022-02-19T01:19:03Z,451337,20,QUARTO TERMO ADITIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6691,6691,20_13.12.2019,REL_ADITAMENTO_CONTRATO,R31,matheus_santos,2022-04-02T20:10:59Z,numero_termo_aditivo,509,matheus_santos,2022-04-02T20:10:54Z,450291,16,3o Termo\nAditivo
6692,6692,19_12.12.2019,REL_ADITAMENTO_CONTRATO,R2,matheus_santos,2022-03-30T20:37:52Z,numero_contrato,129,matheus_santos,2022-02-23T01:30:28Z,314043,8,108/2019
6693,6693,1_9.2.2018.1,REL_AVISO_LICITACAO,R19,tatiana_franco,2022-04-04T01:44:36Z,numero_licitacao,185,tatiana_franco,2022-04-04T01:44:01Z,449518,7,19/2017
6694,6694,16_29.11.2019,REL_ADITAMENTO_CONTRATO,R2,matheus_santos,2022-03-28T21:30:06Z,orgao_contratante,11,matheus_santos,2022-02-22T01:22:40Z,630625,21,BANCO DE BRASILIA S/A


# Limpando para classificar apenas algumas entidades 🧹

Ou seja, apenas: *extrato de contrato, extrato de convênio, aditamento de extrato de contrato, aviso de licitação, aviso de suspensão de licitação, aviso de revogação/anulação de licitação.* 

In [530]:
# retorna dataframe com apenas as entidades passadas no argumento
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]

# extrato de contrato -> EXTRATO_CONTRATO
# extrato de convênio -> EXTRATO_CONVENIO
# aditamento de extrato de contrato -> EXTRATO_ADITAMENTO_CONTRATUAL
# aviso de licitação -> AVISO_LICITACAO
# aviso de suspensão de licitação -> AVISO_SUSPENSAO_LICITACAO
# aviso de revogação/anulação de licitação -> AVISO_ANUL_REV_LICITACAO


df = filter_rows_by_values(raw_df, "tipo_ent", ["EXTRATO_CONTRATO", 
                                                "EXTRATO_CONVENIO",
                                                "EXTRATO_ADITAMENTO_CONTRATUAL",
                                                "AVISO_LICITACAO",
                                                "AVISO_SUSPENSAO_LICITACAO",
                                                "AVISO_ANUL_REV_LICITACAO"])

df


,index,id_dodf,tipo_rel,id_rel,anotador_rel,timestamp_rel,tipo_ent,id_ent,anotador_ent,timestamp_ent,offset,length,texto
1,1,12_22.11.2018,REL_ADITAMENTO_CONTRATO,R15,micael_filipe,2022-04-03T11:46:36Z,EXTRATO_ADITAMENTO_CONTRATUAL,295,micael_filipe,2022-02-22T22:11:01Z,601406,518,EXTRATO DE ADITIVO\n3 Termo Aditivo ao Contrat...
12,12,211_193.9.10.2019,REL_ADITAMENTO_CONTRATO,R7,matheus_santos,2022-04-02T23:00:10Z,EXTRATO_ADITAMENTO_CONTRATUAL,234,matheus_santos,2022-04-02T22:59:51Z,654890,4930,EXTRATO DO PRIMEIRO TERMO ADITIVO DO CONTRATO ...
23,23,5_16.7.2019,REL_ADITAMENTO_CONTRATO,R12,micael_filipe,2022-04-03T12:11:50Z,EXTRATO_ADITAMENTO_CONTRATUAL,195,micael_filipe,2022-04-03T12:11:46Z,454335,836,EXTRATO DE TERMO ADITIVO DO CONTRATO BRB No BR...
30,30,12_22.11.2018,REL_ANUL_REVOG_LICITACAO,R9,lucas_sousa,2022-03-24T13:12:10Z,AVISO_SUSPENSAO_LICITACAO,470,lucas_sousa,2022-03-24T13:12:00Z,530019,510,AVISO DE SUSPENSAO\nCONCORRENCIA No 02/2018\nO...
39,39,5_21.5.2019,REL_ADITAMENTO_CONTRATO,R30,micael_filipe,2022-04-03T11:04:56Z,EXTRATO_ADITAMENTO_CONTRATUAL,403,micael_filipe,2022-04-03T11:04:25Z,376702,437,Especie: Segundo Termo Aditivo ao Contrato no ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6647,6647,211_193.9.10.2019,REL_AVISO_LICITACAO,R16,carla_araujo,2022-04-04T01:12:02Z,AVISO_LICITACAO,283,carla_araujo,2022-04-04T01:11:57Z,648679,863,AVISO DE LICITACAO\nPREGAO ELETRONICO PE No 24...
6651,6651,7_2.8.2019,REL_AVISO_LICITACAO,R22,antonio_carlos,2022-04-03T13:05:42Z,AVISO_LICITACAO,11,antonio_carlos,2022-02-21T18:37:04Z,390033,841,AVISO DE ABERTURA\nPREGAO ELETRONICO POR SRP N...
6658,6658,16_29.11.2019,REL_AVISO_LICITACAO,R19,antonio_carlos,2022-04-03T16:25:31Z,AVISO_LICITACAO,396,antonio_carlos,2022-04-03T16:25:24Z,729101,601,AVISO DE LICITACAO\nLIC ALIENACAO CEB-DIS No 0...
6659,6659,2_25.2.2019,REL_EXTRATO_CONTRATO,R4,alexandre_santana,2022-03-08T00:46:31Z,EXTRATO_CONTRATO,139,alexandre_santana,2022-03-08T00:46:27Z,346995,726,EXTRATO DO CONTRATO No 09/2019\nAQUISICAO DE B...


### Nem sempre é necessária a limpeza total dos dados! 😕

#### ["Aqui entidades numéricas, capitalização e pontuação podem ser importantes. Por isso, pensando em uma tarefa de classificação, faz sentido não adotar alguns dos passos de pré-processamento que normalmente são feitos."](https://colab.research.google.com/drive/1zjcTnNNLlbM3BH3zEAAnQSJjlUmTP8K2?usp=sharing#scrollTo=XP3U0XWKsmSf://)

In [531]:
# classificação em inteiro

# EXTRATO_CONTRATO = 0
# EXTRATO_CONVENIO = 1
# EXTRATO_ADITAMENTO_CONTRATUAL = 2
# AVISO_LICITACAO = 3
# AVISO_SUSPENSAO_LICITACAO = 4
# AVISO_ANUL_REV_LICITACAO = 5

df=df.replace(to_replace="EXTRATO_CONTRATO",value="0")
df=df.replace(to_replace="EXTRATO_CONVENIO",value="1")
df=df.replace(to_replace="EXTRATO_ADITAMENTO_CONTRATUAL",value="2")
df=df.replace(to_replace="AVISO_LICITACAO",value="3")
df=df.replace(to_replace="AVISO_SUSPENSAO_LICITACAO",value="4")
df=df.replace(to_replace="AVISO_ANUL_REV_LICITACAO",value="5")


In [532]:
# 80% vai ser treino, 20% vai ser teste
# 0.8 * 726 = 580
# até o índice 580 vai ser treino, depois disso é teste

# é o dado
x = df.filter(['texto'])

x_train_raw = x.iloc[:580] 
x_test_raw = x.iloc[580:]  

# é o resultado do dado
y = df.filter(['tipo_ent'])

y_train_raw = y.iloc[:580]
y_test_raw = y.iloc[580:]


# preciso transformar o texto num numero

frases = df.filter(['texto'])

valores_distintos = y_train_raw['tipo_ent'].nunique() 
valores_distintos_nao_nulos = y_train_raw['tipo_ent'].count()

print(valores_distintos)              # entidades possíveis
print(valores_distintos_nao_nulos)    # repetições


6
580


### Conversão para "one-hot codes" 🔥

In [533]:
from tensorflow import keras

# conversão para "one-hot codes"

# to_categorical -> vai de 0 até "valores_distintos - 1" como uma possibilidade
# esses dados começam em 1, vou subtrair 1 de tudo, para 0 virar uma categoria

y_train = keras.utils.to_categorical(y_train_raw, valores_distintos)
y_test = keras.utils.to_categorical(y_test_raw, valores_distintos)

print(y_train[0])


[0. 0. 1. 0. 0. 0.]


### Testarei inicialmente a stemnminzação, retirada de acentos (para garantir) e a retirada de stopwords da parte do texto. Na retirada de stopwords, não contarei com a retirada de "\n".

In [534]:
!pip install unidecode
import unidecode

In [535]:
# retirada de acentos

frases['texto'] = frases['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [536]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [537]:
# retirada de stopwords

frases['texto'] = frases['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [538]:
nltk.download('rslp')
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [539]:
# steminização

frases['texto'] = frases['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [540]:
frases = frases.reset_index()
#print(frases['texto'][0])

In [541]:
words_raw = set()

for i in range(726):
    # printing a third element of column
    nossa_string = frases['texto'][i]
    for termo in nossa_string.split():
      words_raw.add(termo)

words_raw = list(words_raw)
num_words = len(words_raw)

def remove_dups(lista):
    """Remove os itens duplicados mantendo a ordem original."""
    return list(dict.fromkeys(lista))

words = remove_dups(words_raw)
words.sort()
num_words = len(words)

# lista de palavras possíveis
num_words

12002

# Mapeando texto em valores numéricos para alimentar a rede neural 🦜

Minha idéia: **Polynomial Rolling Hash**, como explicado neste [link](https://www.geeksforgeeks.org/string-hashing-using-polynomial-rolling-hash-function/).

In [542]:
# Limpeza pronta!

# eu tenho uma lista de palavras, "words"
# agora quero mapear cada palavra em x para um valor inteiro nessa lista


class Hash:
    def __init__(self, s: str):
        self.hash_value = 0
        self.p, self.m = 29791, 10**9 + 7
        self.length = len(s)
        hash_so_far = 0
        p_pow = 1
        for i in range(self.length):
            hash_so_far = (hash_so_far + (1 + ord(s[i]) - ord('a')) * p_pow) % self.m
            p_pow = (p_pow * self.p) % self.m
        self.hash_value = hash_so_far
     
    def __eq__(self, other):
        return self.hash_value == other.hash_value
 


def retorna_valor(frase):

  j = 0
  contador = 0

  acumulado = 0

  for termo in frase.split():
    
    j = Hash(termo)
    acumulado = acumulado + j.hash_value
    contador = contador + 1

  if(contador != 0):
    resultado = acumulado/contador
  else:
    resultado = acumulado

  return resultado

x_train = x_train_raw
x_test = x_test_raw

x_train['texto'] = x_train_raw['texto'].apply(retorna_valor)
x_test['texto'] = x_test_raw['texto'].apply(retorna_valor)

# vamos realizar a visualização do que vai servir de treino
x_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,texto
1,4.493503e+08
12,4.449322e+08
23,5.015193e+08
30,5.067894e+08
39,4.999745e+08
...,...
5264,5.557104e+08
5265,5.279526e+08
5387,4.435323e+08
5394,5.377162e+08


In [543]:
# deve-se mapear o menor e o maior valor para um range entre (x e y)

from scipy.interpolate import interp1d

max_index_train = x_train['texto'].max()
max_index_test = x_test['texto'].max()

if(max_index_train > max_index_test):
  max = max_index_train
else:
  max = max_index_test

min_index_train = x_train['texto'].max()
min_index_test = x_test['texto'].max()

if(min_index_train < min_index_test):
  min = min_index_train
else:
  min = min_index_test

print("Maior valor retornado -> " + str(max))
print("Menor valor retornado -> " + str(min))

def new_map(s):
  a = [min, max]
  b = [0, 4096]
  (a1, a2), (b1, b2) = a, b
  return  b1 + ((s - a1) * (b2 - b1) // (a2 - a1))


x_train['texto'] = x_train['texto'].apply(new_map)
x_test['texto'] = x_test['texto'].apply(new_map)


Maior valor retornado -> 807939627.0
Menor valor retornado -> 695443301.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [544]:
# vamos realizar a visualização do que vai servir de teste

x_test

,texto
5408,-5235.0
5410,-10267.0
5412,-6656.0
5414,-6134.0
5426,-7027.0
...,...
6647,-6693.0
6651,-5014.0
6658,-6451.0
6659,-7219.0


In [545]:
# vamos realizar a visualização do que vai servir de treino

x_train

,texto
1,-8961.0
12,-9122.0
23,-7061.0
30,-6869.0
39,-7118.0
...,...
5264,-5088.0
5265,-6099.0
5387,-9173.0
5394,-5743.0


In [546]:
num_words

12002

In [547]:
print(x_train.shape)
print(x_test.shape)

(580, 1)
(146, 1)


In [548]:
# Definindo shape correto

import numpy 

print(y_train.shape)
#y_train = numpy.delete(y_train, (0), axis=1)
#y_test = numpy.delete(y_test, (0), axis=1)
print(y_train.shape)

(580, 6)
(580, 6)


# RNN: Recurrent Neural Network ♟

### Nesse tipo de implementação, sequência e ordem importam!

#### Qual deveria ser a proporção escolhida perfeita para essa rede neural é uma pergunta complexa.


In [549]:
x_train = np.expand_dims(x_train, axis=-1)
y_train = np.expand_dims(y_train, axis=1)


x_test = np.expand_dims(x_test, axis=-1)
y_test = np.expand_dims(y_test, axis=1)


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(580, 1, 1)
(580, 1, 6)
(146, 1, 1)
(146, 1, 6)


# Criando a rede neural 🧠 

## Dúvida: como deve ser dada a arquitetura da minha rede? 🤔

A rede neural terá uma camada de input com tamanho proporcional à entrada, e uma camada de saída com tamanho proporcional à classificação que será feita. 

Acertar a quantidade de "hidden layers" e quantos nós ela deve ter também é uma tarefa complexa. Dados linearmente separáveis não precisam de uma rede neural com "hidden layer", o que certamente não é este caso. Errar "para mais" não é uma solução otimizada assim como "errar para menos" também não é. 

[Adotarei este cálculo](https://stackoverflow.com/questions/10565868/multi-layer-perceptron-mlp-architecture-criteria-for-choosing-number-of-hidde) para ter um chute decente de quantos nós devo adotar:

(num inputs + num outputs) x 2/3

(726 entradas + 7 classificações) x 2/3 = 489

Arredondando ordens de grandes após alguns testes, farei uma rede com a seguinte estrutura de "hidden layers":

512 x 512 x 512 x 128

In [550]:
#nx_train = x_train.to_numpy()
#ny_train = y_train.to_numpy()
#nx_test = x_test.to_numpy()
#ny_test = y_test.to_numpy()

#nx_train.reshape(1, 5356, 1)
#ny_train.reshape(1, 5356, 625)
#nx_test.reshape(1, 5356, 1)
#zny_test.reshape(1, 5356, 625)

# 28 sequencias de 28 elementos

#x_train = np.expand_dims(x_train, axis=-1)
#y_train = np.expand_dims(y_train, axis=-1)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Input(shape = (T,D)) for RNN
# T = T is length of time sequence
# D = D is the number of features

# x_train.shape[1:] => (1,)

# input_shape=(x_train.shape[1:])
# input_shape=(5356, 1)

model = Sequential()

model.add(LSTM(512, input_shape=(x_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(512, activation='relu', return_sequences=True))
model.add(Dropout(0.1))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(valores_distintos, activation='softmax'))




In [551]:
x_train.shape[1:]

(1, 1)

# Treinamento da RNN 🏋🏻‍♀️ 



In [552]:
print(x_train.shape)
print(y_train.shape)

#x_test = np.expand_dims(x_test, axis=-1)
#y_test = np.expand_dims(y_test, axis=1)

opt = tf.keras.optimizers.Adam(learning_rate=0.1, decay=1e-3)

model.compile(
   loss='categorical_crossentropy',
   optimizer=opt,
   metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs=50,
          validation_data=(x_test, y_test))

(580, 1, 1)
(580, 1, 6)
Epoch 1/50
19/19 [==============================] - 9s 279ms/step - loss: 15836.7451 - accuracy: 0.3810 - val_loss: 1.5054 - val_accuracy: 0.5068
Epoch 2/50
19/19 [==============================] - 3s 171ms/step - loss: 505.3062 - accuracy: 0.5017 - val_loss: 1.2252 - val_accuracy: 0.5068
Epoch 3/50
19/19 [==============================] - 4s 222ms/step - loss: 132.1637 - accuracy: 0.5069 - val_loss: 1.1930 - val_accuracy: 0.5068
Epoch 4/50
19/19 [==============================] - 4s 206ms/step - loss: 1419.3899 - accuracy: 0.5086 - val_loss: 1.1956 - val_accuracy: 0.5068
Epoch 5/50
19/19 [==============================] - 3s 183ms/step - loss: 761.6019 - accuracy: 0.5069 - val_loss: 1.1958 - val_accuracy: 0.5068
Epoch 6/50
19/19 [==============================] - 2s 111ms/step - loss: 1077.1671 - accuracy: 0.5086 - val_loss: 1.1955 - val_accuracy: 0.5068
Epoch 7/50
19/19 [==============================] - 2s 111ms/step - loss: 359.4043 - accuracy: 0.5103 - val_

# Avaliação do modelo 🤙🏻

In [553]:
score = model.evaluate(x_test, y_test, verbose=0)


# Resultados 😳


In [554]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.2048616409301758
Test accuracy: 0.5068492889404297


### Esse é o melhor resultado que fui capaz de atingir e acredito estar satisfatório, comparando com o resultado prévio de classificação de todos as entidades.